In [2]:
# 임베딩 + 데이터 결합
import pandas as pd

# 파일 경로 정의
VECTOR_CSV_PATH = r"C:\Users\user\developments\hk-final\db\news_v2_vector_202506122113.csv"
RAW_CSV_PATH = r"C:\\Users\\user\\developments\\hk-final\\db\\news_2023_2025_raw.csv"
OUTPUT_CSV_PATH = r"C:\\Users\\user\\developments\\hk-final\\db\\merged_news_data.csv" # 병합된 데이터를 저장할 파일 경로

# 1. 뉴스 데이터 로드
try:
    df_vector = pd.read_csv(VECTOR_CSV_PATH)
    df_raw = pd.read_csv(RAW_CSV_PATH)
except FileNotFoundError as e:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {e}")
    exit() # 파일이 없으면 스크립트 종료

print(f"'{VECTOR_CSV_PATH}' 로드 완료. 행 수: {len(df_vector)}, 컬럼: {df_vector.columns.tolist()}")
print(f"'{RAW_CSV_PATH}' 로드 완료. 행 수: {len(df_raw)}, 컬럼: {df_raw.columns.tolist()}")

# 2. news_id를 기준으로 두 데이터프레임 병합
# df_vector: news_id, embedding 컬럼
# df_raw: news_id, wdate, title, article, press, url, image 컬럼
# 'inner' 조인을 사용하여 양쪽에 모두 존재하는 news_id만 병합합니다.
merged_df = pd.merge(df_vector, df_raw, on='news_id', how='inner')

print(f"\n데이터 병합 완료. 최종 행 수: {len(merged_df)}, 컬럼: {merged_df.columns.tolist()}")
print("\n병합된 데이터의 상위 5개 행:")
print(merged_df.head())

# 3. 병합된 데이터를 CSV 파일로 저장 (선택 사항)
# 다음 단계에서 이 병합된 파일을 바로 사용할 것이라면 굳이 저장할 필요는 없습니다.
# 하지만 중간 결과를 확인하거나 재사용해야 한다면 저장하는 것이 좋습니다.
try:
    merged_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"\n병합된 데이터가 '{OUTPUT_CSV_PATH}'에 저장되었습니다.")
except Exception as e:
    print(f"오류: 병합된 데이터 저장 중 문제가 발생했습니다: {e}")

'C:\Users\user\developments\hk-final\db\news_v2_vector_202506122113.csv' 로드 완료. 행 수: 14087, 컬럼: ['news_id', 'embedding']
'C:\\Users\\user\\developments\\hk-final\\db\\news_2023_2025_raw.csv' 로드 완료. 행 수: 14087, 컬럼: ['news_id', 'wdate', 'title', 'article', 'press', 'url', 'image']

데이터 병합 완료. 최종 행 수: 14087, 컬럼: ['news_id', 'embedding', 'wdate', 'title', 'article', 'press', 'url', 'image']

병합된 데이터의 상위 5개 행:
         news_id                                          embedding  \
0  20240812_0049  [-0.6289959,-0.41044942,-0.21019596,-0.2123955...   
1  20240812_0051  [0.35456046,-0.9542539,-0.41639727,-0.02229850...   
2  20240610_0241  [-1.1654333,-0.32814464,-0.3500396,-0.22474843...   
3  20240610_0245  [0.7812012,-0.8581933,0.033431143,0.5623036,0....   
4  20240610_0246  [-0.16374397,-0.6875845,0.10508025,-0.567515,0...   

                 wdate                                   title  \
0  2024-08-12 16:22:00                NH·미래에셋, 증권사 시총 1위 놓고 접전   
1  2024-08-12 16:21:00  “주가는 언제 

In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_id    14087 non-null  object
 1   embedding  14087 non-null  object
 2   wdate      14087 non-null  object
 3   title      14087 non-null  object
 4   article    14087 non-null  object
 5   press      14087 non-null  object
 6   url        14087 non-null  object
 7   image      14087 non-null  object
dtypes: object(8)
memory usage: 880.6+ KB


In [6]:
merged_df = merged_df.drop("image", axis=1)

In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_id    14087 non-null  object
 1   embedding  14087 non-null  object
 2   wdate      14087 non-null  object
 3   title      14087 non-null  object
 4   article    14087 non-null  object
 5   press      14087 non-null  object
 6   url        14087 non-null  object
dtypes: object(7)
memory usage: 770.5+ KB


In [8]:
merged_df.to_csv("C:\\Users\\user\\developments\\hk-final\\db\\merged_news_data.csv", index=False)

In [ ]:
import pandas as pd
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # 또는 사용하시는 임베딩 모델

# 1. 데이터 로드 (예시 CSV 파일, 실제 경로로 변경 필요)
df = pd.read_csv("db\merged_news_data.csv") # 여기에 실제 CSV 파일 경로를 넣어주세요.

# 2. Document 객체 생성
documents = []
for _, row in df.iterrows():
    doc = Document(
        page_content=row['article'],  # 임베딩할 실제 텍스트
        metadata={
            "wdate": row['wdate'],
            "title": row['title'],
            "press": row['press'],
            "url": row['url']
        }
    )
    documents.append(doc)

# 3. 임베딩 모델 로드 (FAISS 저장 시 필요)
# 임베딩 모델을 직접 로드해야 FAISS가 문서를 임베딩하여 저장할 수 있습니다.
# 예시: HuggingFaceEmbeddings
# model_name = "BAAI/bge-small-en-v1.5" # 사용하실 임베딩 모델 명
# embedding_model = HuggingFaceEmbeddings(model_name=model_name)

# ⭐️ 중요: 여기서는 외부 API를 통해 임베딩을 수행한다고 하셨으므로,
# 실제 FAISS를 생성할 때는 해당 임베딩 API를 호출하여 벡터를 미리 생성한 후
# FAISS.from_embeddings()를 사용하는 것이 효율적입니다.
# 만약 FAISS.from_documents()를 사용한다면, FAISS 내부적으로 embedding_model을 사용하여 임베딩합니다.
# 여기서는 편의상 FAISS.from_documents()와 로컬 임베딩 모델을 사용한다고 가정합니다.
# 실제 운영 환경에서는 아래 FAISS.from_documents() 대신,
# 외부 임베딩 API로 문서들을 미리 임베딩하여 (text, vector) 튜플 리스트를 만든 후
# FAISS.from_embeddings(text_embeddings=your_pre_embedded_data, embedding=None)
# 와 같이 사용하는 것을 고려해보세요.

# 현재 코드는 FAISS.from_documents()를 사용하며, 이 경우 embedding_model이 필수입니다.
# 실제 임베딩 API 호출을 통해 벡터를 미리 생성하는 방식으로 변경하려면 추가적인 코드 수정이 필요합니다.
# 여기서는 FAISS가 임베딩 모델을 사용하여 저장할 수 있도록 embedding_model을 전달합니다.
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5") # 예시 임베딩 모델

# 4. 벡터 DB (FAISS) 생성 및 저장
vector_db = FAISS.from_documents(documents, embedding_model)

# 5. 벡터 DB 저장
vector_db_path = "vectorstore/news_db"
vector_db.save_local(vector_db_path)
print(f"FAISS 벡터 DB가 '{vector_db_path}'에 성공적으로 저장되었습니다.")